PART2-Handling with Missing Values

In [194]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from pylab import rcParams
sns.set_style('darkgrid')
import re
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [195]:
df = pd.read_csv("clean_scout.csv")

In [196]:
df.shape

(15919, 32)

In [197]:
df.duplicated().sum()

1676

In [198]:
df[df.duplicated()].sort_values(by=['make_model','body_type','price','km']).head(20)

,make_model,body_type,price,vat,km,Type,Warranty,Cylinders,Fuel,Comfort_Convenience,...,Nr_of_Seats,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb,cons_city,cons_country,CO2_Emission,Emission_Class
337,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
339,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
340,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
341,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
342,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
343,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
344,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
345,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
346,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6
349,Audi A1,Compact,14990,NaN,49351.00,Used,NaN,NaN,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,NaN,NaN,4.00,4.40,3.70,104.00,Euro 6


In [199]:
df.head(3).T

,0,1,2
make_model,Audi A1,Audi A1,Audi A1
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,56013.00,80000.00,83450.00
Type,Used,Used,Used
Warranty,NaN,NaN,NaN
Cylinders,3.00,4.00,NaN
Fuel,Diesel,Benzine,Diesel
Comfort_Convenience,"Air conditioning,Armrest,Automatic climate con...","Air conditioning,Automatic climate control,Hil...","Air conditioning,Cruise control,Electrical sid..."


In [200]:
df.head(3)

,make_model,body_type,price,vat,km,Type,Warranty,Cylinders,Fuel,Comfort_Convenience,...,Nr_of_Seats,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb,cons_city,cons_country,CO2_Emission,Emission_Class
0,Audi A1,Sedans,15770,VAT deductible,56013.00,Used,NaN,3.00,Diesel,"Air conditioning,Armrest,Automatic climate con...",...,5.00,Automatic,1422.00,1220.00,front,3.80,4.30,3.50,99.00,Euro 6
1,Audi A1,Sedans,14500,Price negotiable,80000.00,Used,NaN,4.00,Benzine,"Air conditioning,Automatic climate control,Hil...",...,4.00,Automatic,1798.00,1255.00,front,5.60,7.10,4.70,129.00,Euro 6
2,Audi A1,Sedans,14640,VAT deductible,83450.00,Used,NaN,NaN,Diesel,"Air conditioning,Cruise control,Electrical sid...",...,4.00,Automatic,1598.00,NaN,front,3.80,4.40,3.40,99.00,Euro 6


In [201]:
df.isnull().sum()*100 / df.shape[0]

make_model             0.00
body_type              0.38
price                  0.00
vat                   28.35
km                     6.43
Type                   0.01
Warranty              69.51
Cylinders             35.68
Fuel                   0.00
Comfort_Convenience    5.78
Entertainment_Media    8.63
Extras                18.61
Safety_Security        6.17
Gears                 29.60
age                   10.03
Previous_Owners       41.71
hp_kW                  0.55
Inspection_new        75.30
Paint_Type            36.26
Upholstery_type       30.60
Upholstery_color      31.90
Nr_of_Doors            1.33
Nr_of_Seats            6.14
Gearing_Type           0.00
Displacement_cc        3.12
Weight_kg             43.81
Drive_chain           43.08
cons_comb             12.77
cons_city             15.30
cons_country          14.93
CO2_Emission          15.30
Emission_Class        22.79
dtype: float64

In [202]:
miss_val = []

[miss_val.append(column) for column in df.columns if any(df[column].isnull())]

print("Columns that have missing values :")

miss_val

Columns that have missing values :


['body_type',
 'vat',
 'km',
 'Type',
 'Warranty',
 'Cylinders',
 'Comfort_Convenience',
 'Entertainment_Media',
 'Extras',
 'Safety_Security',
 'Gears',
 'age',
 'Previous_Owners',
 'hp_kW',
 'Inspection_new',
 'Paint_Type',
 'Upholstery_type',
 'Upholstery_color',
 'Nr_of_Doors',
 'Nr_of_Seats',
 'Displacement_cc',
 'Weight_kg',
 'Drive_chain',
 'cons_comb',
 'cons_city',
 'cons_country',
 'CO2_Emission',
 'Emission_Class']

In [203]:
def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print("--------------------------------")
    print(df[col].value_counts(dropna = False))

functions to fill the missing value

In [204]:
def fill_most_freq(df, group_col, col_name):
    
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [205]:
def fill_prop(df, group_col, col_name):
    
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [206]:
def fill(df, group_col1, group_col2, col_name, method): 
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [207]:
#yukaridaki 3 def her yerde kullanilabilir iyi korumak gerek!!!!!!!!
# bu kodlari mean,median ya da mode a göre missing value doldurmak icin kullanabiliriz

age

In [208]:
first_looking("age")

column name    :  age
--------------------------------
per_of_nulls   :  % 10.03
num_of_nulls   :  1597
num_of_uniques :  4
--------------------------------
1.00    4522
3.00    3674
2.00    3273
0.00    2853
NaN     1597
Name: age, dtype: int64


In [209]:
df['age'].fillna('-', inplace=True)

In [210]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
-      1597
Name: age, dtype: int64

In [211]:
df.groupby("age").km.describe()    #934 mean degerinin 0 a yakin oldugunu görüyoruz, min 0 bu sonucu elde etmek icin age sütunundaki nan lari bir sekilde tanimliyoruz, 
                                    #biz '- ' isareti kullandik

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,2706.00,2085.36,5365.88,1.00,10.00,50.00,3000.00,127022.00
1.0,4484.00,18035.24,11052.52,1.00,9990.00,17872.00,25078.50,136000.00
2.0,3272.00,41754.94,28295.75,1.00,21541.75,34752.00,54805.50,317000.00
3.0,3674.00,77442.52,39170.14,10.00,48000.00,72914.50,99950.00,291800.00
-,759.00,934.50,7416.24,0.00,5.00,10.00,10.00,89982.00


In [212]:
df[df["age"] == "-"]["km"].value_counts(dropna=False)  #yas arttikca km de artar, nanlari km özelinde getirdik 

NaN         838
10.00       369
1.00        146
5.00         58
20.00        32
15.00        21
0.00         19
11.00        12
8.00         11
50.00        10
12.00         8
100.00        8
7.00          7
3.00          4
9.00          4
4.00          3
250.00        3
25.00         3
30.00         3
3000.00       2
39962.00      2
2.00          2
22627.00      2
784.00        1
89692.00      1
3500.00       1
325.00        1
497.00        1
99.00         1
77.00         1
40.00         1
19500.00      1
6100.00       1
11000.00      1
89982.00      1
4307.00       1
141.00        1
34164.00      1
500.00        1
150.00        1
11200.00      1
20768.00      1
32084.00      1
142.00        1
81800.00      1
281.00        1
6.00          1
68485.00      1
85000.00      1
196.00        1
4500.00       1
60.00         1
5000.00       1
Name: km, dtype: int64

In [213]:
cond1 = (df['km'] < 10000)
cond2 = ((df['km'] >= 10000) & (df['km'] < 28000))
cond3 = ((df['km'] >= 28000) & (df['km'] < 50000))
cond4 = (df['km'] >= 50000)

In [214]:
#km si 10000 altindaki - degerleri 0 yap......

In [215]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('-', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('-', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('-', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('-', 3)

In [216]:
df.groupby('age').km.mean()

age
0.0    1647.36
1.0   18035.13
2.0   41748.58
3.0   77450.06
-          NaN
Name: km, dtype: float64

In [217]:
df["age"].value_counts(dropna=False)  #hala 838 - kalmis

1.0    4528
3.0    3679
0.0    3597
2.0    3277
-       838
Name: age, dtype: int64

In [218]:
df.groupby(['make_model',"body_type", 'age']).price.describe() #hala - yerine doldurulmayanlari groupby yapiyoruz

count     mean     std      min      25%  \
make_model     body_type age                                             
Audi A1        Compact   0.0 198.00 23277.43 3510.41 14900.00 20503.50   
                         1.0 268.00 18596.04 2659.91 13980.00 16445.00   
                         2.0 161.00 16602.81 2085.38 10999.00 15450.00   
                         3.0 234.00 14532.91 1908.91  9950.00 13407.50   
                         -   178.00 23996.26 3383.85 16220.00 21515.00   
...                             ...      ...     ...      ...      ...   
Renault Espace Van       0.0 218.00 40074.51 8154.73 24200.00 32975.00   
                         1.0 207.00 31947.84 4072.97 23099.00 29385.00   
                         2.0 134.00 26574.48 3427.86 14600.00 24982.50   
                         3.0 199.00 22132.62 3216.03 12614.00 19900.00   
                         -    19.00 39632.32 6866.14 28650.00 33848.00   

                                  50%      75%      max  
make_model     body_type age                             
Audi A1        Compact   0.0 22492.00 26798.50 31990.00  
                         1.0 16980.00 20950.00 23829.00  
                         2.0 15850.00 17700.00 22150.00  
                         3.0 13994.50 15480.00 18900.00  
                         -   22875.00 27380.00 29181.00  
...                               ...      ...      ...  
Renault Espace Van       0.0 38925.00 45667.25 68320.00  
                         1.0 31900.00 34990.00 38990.00  
                         2.0 25490.00 28500.00 33800.00  
                         3.0 22222.00 24650.00 27900.00  
                         -   39550.00 45950.50 50170.00  

[176 rows x 8 columns]

In [219]:
df['age'].replace('-', 0, inplace=True)     # - ile olan araclar 0 yas gibi,o yüzden oraya ekledik

In [220]:
df.groupby('age').km.mean()   

age
0.00    1647.36
1.00   18035.13
2.00   41748.58
3.00   77450.06
Name: km, dtype: float64

In [221]:
df["age"].value_counts(dropna=False)

1.00    4528
0.00    4435
3.00    3679
2.00    3277
Name: age, dtype: int64

km

In [222]:
first_looking("km")

column name    :  km
--------------------------------
per_of_nulls   :  % 6.43
num_of_nulls   :  1024
num_of_uniques :  6689
--------------------------------
10.00       1045
NaN         1024
1.00         367
5.00         170
50.00        148
            ... 
67469.00       1
43197.00       1
10027.00       1
35882.00       1
57.00          1
Name: km, Length: 6690, dtype: int64


In [223]:
df.groupby("age").km.mean()

age
0.00    1647.36
1.00   18035.13
2.00   41748.58
3.00   77450.06
Name: km, dtype: float64

In [224]:
df[df["km"].isna()][["age", "km"]]

,age,km
743,0.00,NaN
869,0.00,NaN
946,0.00,NaN
977,0.00,NaN
980,0.00,NaN
...,...,...
15890,0.00,NaN
15902,0.00,NaN
15912,0.00,NaN
15914,0.00,NaN


In [225]:
df.groupby("age").km.transform("mean")    #string function kullaninca transform ayni uzunlukta data frame döndürür.(age uzunlugunda)
                                         #mesela 15914 1647 ile dolduruldu,yasi 0 a yakin

0       77450.06
1       41748.58
2       77450.06
3       77450.06
4       77450.06
          ...   
15914    1647.36
15915    1647.36
15916    1647.36
15917    1647.36
15918    1647.36
Name: km, Length: 15919, dtype: float64

In [226]:
df["km"].fillna(df.groupby("age").km.transform("mean"), inplace=True)

In [227]:
df.km.value_counts(dropna=False)

10.00       1045
1647.36      985
1.00         367
5.00         170
50.00        148
            ... 
67469.00       1
43197.00       1
10027.00       1
35882.00       1
57.00          1
Name: km, Length: 6692, dtype: int64

body_type

In [228]:
first_looking("body_type")

column name    :  body_type
--------------------------------
per_of_nulls   :  % 0.38
num_of_nulls   :  60
num_of_uniques :  9
--------------------------------
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


In [229]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [230]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [231]:
df["body_type"].mode()

0    Sedans
Name: body_type, dtype: object

In [232]:
df["body_type"].mode()[0]   #bazen 2 deger gelir o yüzden mode kacinciyi alsin diye belirtmeliyiz
                            # mode en fazla tekrar eden demek diye onu alir ama uygun degil o yüzden kendimiz bir cond tanimlamaliyiz

'Sedans'

In [233]:
for group in list(df["make_model"].unique()):
    cond = df["make_model"]==group
    mode = list(df[cond]["body_type"].mode())
    if mode != []:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])
    else:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df["body_type"].mode()[0])

In [234]:
df['body_type'].value_counts(dropna=False)

Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [235]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [236]:
fill_most(df, "make_model", "body_type")   #fill_most bizim mode umuzdu

Number of NaN :  0
------------------
Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


Previous_Owners

In [237]:
first_looking("Previous_Owners")

column name    :  Previous_Owners
--------------------------------
per_of_nulls   :  % 41.71
num_of_nulls   :  6640
num_of_uniques :  5
--------------------------------
1.00    8294
NaN     6640
2.00     778
0.00     188
3.00      17
4.00       2
Name: Previous_Owners, dtype: int64


In [238]:
df["Previous_Owners"].fillna("-", inplace = True)

In [239]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [240]:
df.groupby(['make_model', 'age', 'Previous_Owners']).km.describe()

count     mean      std      min  \
make_model     age  Previous_Owners                                     
Audi A1        0.00 0.0              45.00   958.37   815.36     0.00   
                    1.0             239.00  3069.54  3460.40     1.00   
                    2.0               1.00  3000.00      NaN  3000.00   
                    -               521.00  1549.99  2303.80     1.00   
               1.00 0.0               1.00 15000.00      NaN 15000.00   
...                                    ...      ...      ...      ...   
Renault Espace 3.00 1.0             118.00 79365.81 36296.88  3532.00   
                    2.0              20.00 55316.25 27356.18  7300.00   
                    3.0               1.00 40000.00      NaN 40000.00   
                    4.0               1.00 57000.00      NaN 57000.00   
                    -               159.00 73332.06 32702.20  6119.00   

                                         25%      50%      75%       max  
make_model     age  Previous_Owners                                       
Audi A1        0.00 0.0                10.00  1647.36  1647.36   1647.36  
                    1.0                20.00  2500.00  5000.00  18000.00  
                    2.0              3000.00  3000.00  3000.00   3000.00  
                    -                  11.00  1647.36  1647.36  15500.00  
               1.00 0.0             15000.00 15000.00 15000.00  15000.00  
...                                      ...      ...      ...       ...  
Renault Espace 3.00 1.0             55000.00 76475.00 98375.75 197414.00  
                    2.0             42536.75 51972.50 64921.25 148000.00  
                    3.0             40000.00 40000.00 40000.00  40000.00  
                    4.0             57000.00 57000.00 57000.00  57000.00  
                    -               50027.00 68570.00 94000.00 169068.00  

[120 rows x 8 columns]

In [241]:
df[(df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")]["km"]

14894   1647.36
14895   1647.36
14896    101.00
14897   1647.36
14898    101.00
14899    101.00
14900   1647.36
14901    101.00
14903    101.00
14904   1647.36
14905   1647.36
14906    101.00
14907    101.00
14908   1647.36
14909   1647.36
14910    101.00
14911   1647.36
14912    101.00
14913   1647.36
14914    101.00
14915    101.00
14916    101.00
14917   1647.36
14918   1647.36
14919   1647.36
14920    101.00
14921   1647.36
14922   1647.36
14923    101.00
14924   1647.36
14925    101.00
14926    101.00
14927   1647.36
Name: km, dtype: float64

In [242]:
cond = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")   #34 tane renault var 33 tane nan var ,ortalama km 980lerde
df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].replace("-", 0.0)

In [243]:
df["Previous_Owners"].value_counts(dropna=False)   #6607 tane hala nan var

1.0    8294
-      6607
2.0     778
0.0     221
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [244]:
df["Previous_Owners"].replace("-", np.nan, inplace=True)

In [245]:
for group in list(df["age"].unique()):
    cond = df["age"]==group
    df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].fillna(method="ffill").fillna(method="bfill")
df["Previous_Owners"] = df["Previous_Owners"].fillna(method="ffill").fillna(method="bfill")

In [246]:
df["Previous_Owners"].value_counts(dropna=False)   #hicbir nan deger kalmadi

1.00    14162
2.00     1172
0.00      554
3.00       29
4.00        2
Name: Previous_Owners, dtype: int64

In [247]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [248]:
fill_prop(df, "age", "Previous_Owners")

Number of NaN :  0
------------------
1.00    14162
2.00     1172
0.00      554
3.00       29
4.00        2
Name: Previous_Owners, dtype: int64


Warranty

In [249]:
first_looking("Warranty")  #nanlar %69.51

column name    :  Warranty
--------------------------------
per_of_nulls   :  % 69.51
num_of_nulls   :  11066
num_of_uniques :  41
--------------------------------
NaN      11066
12.00     2594
24.00     1118
60.00      401
36.00      279
48.00      149
6.00       125
72.00       59
3.00        33
23.00       11
18.00       10
20.00        7
25.00        6
2.00         5
50.00        4
26.00        4
16.00        4
4.00         3
1.00         3
19.00        3
34.00        3
13.00        3
28.00        2
22.00        2
14.00        2
11.00        2
46.00        2
21.00        2
9.00         2
17.00        2
45.00        2
33.00        1
40.00        1
65.00        1
10.00        1
15.00        1
7.00         1
8.00         1
56.00        1
49.00        1
47.00        1
30.00        1
Name: Warranty, dtype: int64


In [250]:
df["Warranty"].fillna("-", inplace = True)

In [251]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

count     mean     std      min      25%  \
make_model     age  Warranty                                             
Audi A1        0.00 12.0      38.00 23846.34 2978.85 17900.00 21000.00   
                    18.0       1.00 19995.00     NaN 19995.00 19995.00   
                    24.0     107.00 23766.31 3338.11 14900.00 21890.00   
                    36.0      26.00 25514.42 3205.84 19900.00 22489.25   
                    48.0      18.00 24441.67 2516.78 19650.00 22800.00   
...                             ...      ...     ...      ...      ...   
Renault Espace 3.00 24.0       4.00 24906.25  827.18 23890.00 24535.00   
                    40.0       1.00 22500.00     NaN 22500.00 22500.00   
                    48.0       1.00 19490.00     NaN 19490.00 19490.00   
                    60.0       1.00 23890.00     NaN 23890.00 23890.00   
                    -        161.00 21931.56 3562.32 12614.00 19013.00   

                                  50%      75%      max  
make_model     age  Warranty                             
Audi A1        0.00 12.0     23650.00 25725.00 28990.00  
                    18.0     19995.00 19995.00 19995.00  
                    24.0     22900.00 26890.00 32000.00  
                    36.0     26400.00 28677.50 29179.00  
                    48.0     25300.00 26200.00 28500.00  
...                               ...      ...      ...  
Renault Espace 3.00 24.0     24925.00 25296.25 25885.00  
                    40.0     22500.00 22500.00 22500.00  
                    48.0     19490.00 19490.00 19490.00  
                    60.0     23890.00 23890.00 23890.00  
                    -        21900.00 24899.00 30000.00  

[259 rows x 8 columns]

In [252]:
df.groupby(["make_model", "Warranty"]).price.describe().sort_values(by = ["make_model", "mean"], ascending=False)

count     mean     std      min      25%      50%  \
make_model     Warranty                                                      
Renault Espace 48.00     14.00 39806.79 8058.50 19490.00 37035.00 40692.50   
               16.00      1.00 38890.00     NaN 38890.00 38890.00 38890.00   
               45.00      1.00 37950.00     NaN 37950.00 37950.00 37950.00   
               23.00      1.00 37600.00     NaN 37600.00 37600.00 37600.00   
               25.00      1.00 36498.00     NaN 36498.00 36498.00 36498.00   
...                        ...      ...     ...      ...      ...      ...   
Audi A1        23.00      2.00 16560.00 6208.40 12170.00 14365.00 16560.00   
               47.00      1.00 16370.00     NaN 16370.00 16370.00 16370.00   
               49.00      1.00 16270.00     NaN 16270.00 16270.00 16270.00   
               46.00      1.00 15980.00     NaN 15980.00 15980.00 15980.00   
               26.00      1.00 15840.00     NaN 15840.00 15840.00 15840.00   

                             75%      max  
make_model     Warranty                    
Renault Espace 48.00    46095.00 47990.00  
               16.00    38890.00 38890.00  
               45.00    37950.00 37950.00  
               23.00    37600.00 37600.00  
               25.00    36498.00 36498.00  
...                          ...      ...  
Audi A1        23.00    18755.00 20950.00  
               47.00    16370.00 16370.00  
               49.00    16270.00 16270.00  
               46.00    15980.00 15980.00  
               26.00    15840.00 15840.00  

[130 rows x 8 columns]

In [253]:
df.drop("Warranty", axis=1, inplace=True) #cok fazla nan value var,saglikli data yok o yüzden warranty drop ediliyor

In [254]:
df.shape

(15919, 31)

vat

In [255]:
first_looking("vat")

column name    :  vat
--------------------------------
per_of_nulls   :  % 28.35
num_of_nulls   :  4513
num_of_uniques :  2
--------------------------------
VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64


In [256]:
df["vat"].fillna("-", inplace = True)

In [257]:
df.groupby(["make_model", "body_type", "vat"]).price.describe()

count     mean     std      min  \
make_model     body_type   vat                                                 
Audi A1        Compact     -                257.00 16692.86 3377.35 11100.00   
                           Price negotiable   3.00 17631.67 1548.95 15950.00   
                           VAT deductible   779.00 20019.00 4603.19  9950.00   
               Coupe       VAT deductible     2.00 14925.00 1378.86 13950.00   
               Sedans      -                498.00 17650.87 4225.47  8999.00   
...                                            ...      ...     ...      ...   
Renault Espace Transporter Price negotiable   7.00 21500.00 3964.85 19000.00   
                           VAT deductible    25.00 22685.48 4915.59 14999.00   
               Van         -                308.00 27953.37 6653.75 13500.00   
                           Price negotiable  19.00 25957.79 3789.47 20999.00   
                           VAT deductible   484.00 32989.40 9368.43 12614.00   

                                                 25%      50%      75%  \
make_model     body_type   vat                                           
Audi A1        Compact     -                14990.00 15850.00 17900.00   
                           Price negotiable 16947.50 17945.00 18472.50   
                           VAT deductible   16370.00 19990.00 22730.00   
               Coupe       VAT deductible   14437.50 14925.00 15412.50   
               Sedans      -                14819.00 16495.00 20300.00   
...                                              ...      ...      ...   
Renault Espace Transporter Price negotiable 19050.00 19900.00 21750.00   
                           VAT deductible   19450.00 20900.00 27000.00   
               Van         -                23867.50 26224.50 32042.50   
                           Price negotiable 22300.00 26950.00 28250.00   
                           VAT deductible   25990.00 31975.00 38110.00   

                                                 max  
make_model     body_type   vat                        
Audi A1        Compact     -                29181.00  
                           Price negotiable 19000.00  
                           VAT deductible   31990.00  
               Coupe       VAT deductible   15900.00  
               Sedans      -                37900.00  
...                                              ...  
Renault Espace Transporter Price negotiable 30000.00  
                           VAT deductible   32700.00  
               Van         -                49390.00  
                           Price negotiable 32000.00  
                           VAT deductible   68320.00  

[99 rows x 8 columns]

In [258]:
df.drop("vat", axis=1, inplace=True)

In [259]:
df.shape

(15919, 30)

Paint_Type

In [260]:
first_looking("Paint_Type")  #elde edilen bilgilere göre paint_type price i pek etkilemiyor

column name    :  Paint_Type
--------------------------------
per_of_nulls   :  % 36.26
num_of_nulls   :  5772
num_of_uniques :  3
--------------------------------
Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64


In [261]:
df["Paint_Type"].fillna("-", inplace = True)

In [262]:
df.groupby(["make_model", "body_type", "age", 'Paint_Type']).price.describe()

count     mean     std      min  \
make_model     body_type age  Paint_Type                                    
Audi A1        Compact   0.00 -          145.00 22082.72 3341.75 14900.00   
                              Metallic   229.00 24622.26 3172.06 17880.00   
                              Uni/basic    2.00 19888.00    0.00 19888.00   
                         1.00 -           56.00 17590.52 2369.31 14220.00   
                              Metallic   212.00 18861.65 2674.02 13980.00   
...                                         ...      ...     ...      ...   
Renault Espace Van       2.00 Metallic    91.00 26799.30 3337.26 16950.00   
                              Uni/basic    1.00 29200.00     NaN 29200.00   
                         3.00 -           67.00 21713.15 3450.67 12990.00   
                              Metallic   136.00 22317.03 3286.73 12614.00   
                              Uni/basic    3.00 22628.33 2094.72 20995.00   

                                              25%      50%      75%      max  
make_model     body_type age  Paint_Type                                      
Audi A1        Compact   0.00 -          19850.00 21490.00 23475.00 29181.00  
                              Metallic   21990.00 23700.00 27780.00 31990.00  
                              Uni/basic  19888.00 19888.00 19888.00 19888.00  
                         1.00 -          15852.25 16864.50 18367.25 23650.00  
                              Metallic   16448.75 17515.00 21482.50 23829.00  
...                                           ...      ...      ...      ...  
Renault Espace Van       2.00 Metallic   25490.00 25490.00 28495.00 33800.00  
                              Uni/basic  29200.00 29200.00 29200.00 29200.00  
                         3.00 -          19195.00 22900.00 23490.00 32900.00  
                              Metallic   19900.00 21990.00 24912.50 27900.00  
                              Uni/basic  21447.50 21900.00 23445.00 24990.00  

[272 rows x 8 columns]

In [263]:
df.groupby(["make_model", "body_type", 'Paint_Type']).price.describe().sort_values(by = ["make_model", "body_type", "mean"], ascending = False)

count     mean     std      min  \
make_model     body_type   Paint_Type                                    
Renault Espace Van         Metallic   505.00 31507.81 9533.34 12614.00   
                           -          293.00 30015.82 7147.55 12990.00   
                           Uni/basic   13.00 27971.15 4495.66 20995.00   
               Transporter Metallic    34.00 22865.82 4255.81 14999.00   
                           -           18.00 22116.61 4248.69 16900.00   
...                                      ...      ...     ...      ...   
Audi A1        Sedans      -          493.00 18104.02 4529.70  8999.00   
               Coupe       Metallic     2.00 14925.00 1378.86 13950.00   
               Compact     Metallic   723.00 19412.38 4665.03  9950.00   
                           -          311.00 18709.10 4273.17 10490.00   
                           Uni/basic    5.00 16815.20 3405.16 12900.00   

                                           25%      50%      75%      max  
make_model     body_type   Paint_Type                                      
Renault Espace Van         Metallic   24950.00 29500.00 36781.00 68320.00  
                           -          24790.00 30965.00 34070.00 54205.00  
                           Uni/basic  24750.00 26900.00 31900.00 34000.00  
               Transporter Metallic   19924.75 21750.00 24975.00 31000.00  
                           -          19575.00 20750.00 25250.00 32700.00  
...                                        ...      ...      ...      ...  
Audi A1        Sedans      -          14300.00 17300.00 20990.00 37900.00  
               Coupe       Metallic   14437.50 14925.00 15412.50 15900.00  
               Compact     Metallic   15740.00 18750.00 22490.00 31990.00  
                           -          15850.00 17950.00 21490.00 29181.00  
                           Uni/basic  13500.00 17900.00 19888.00 19888.00  

[98 rows x 8 columns]

In [264]:
df["Paint_Type"].replace("-", np.nan, inplace=True)

In [265]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64

In [266]:
df.groupby(["make_model", "body_type", "age", 'Paint_Type']).price.describe()

count     mean     std      min  \
make_model     body_type age  Paint_Type                                    
Audi A1        Compact   0.00 Metallic   229.00 24622.26 3172.06 17880.00   
                              Uni/basic    2.00 19888.00    0.00 19888.00   
                         1.00 Metallic   212.00 18861.65 2674.02 13980.00   
                         2.00 Metallic   108.00 17001.35 2294.78 10999.00   
                              Uni/basic    1.00 17900.00     NaN 17900.00   
...                                         ...      ...     ...      ...   
Renault Espace Van       1.00 Uni/basic    8.00 30292.50 3571.28 24750.00   
                         2.00 Metallic    91.00 26799.30 3337.26 16950.00   
                              Uni/basic    1.00 29200.00     NaN 29200.00   
                         3.00 Metallic   136.00 22317.03 3286.73 12614.00   
                              Uni/basic    3.00 22628.33 2094.72 20995.00   

                                              25%      50%      75%      max  
make_model     body_type age  Paint_Type                                      
Audi A1        Compact   0.00 Metallic   21990.00 23700.00 27780.00 31990.00  
                              Uni/basic  19888.00 19888.00 19888.00 19888.00  
                         1.00 Metallic   16448.75 17515.00 21482.50 23829.00  
                         2.00 Metallic   15450.00 15954.50 18957.50 22150.00  
                              Uni/basic  17900.00 17900.00 17900.00 17900.00  
...                                           ...      ...      ...      ...  
Renault Espace Van       1.00 Uni/basic  26900.00 31900.00 32492.50 34000.00  
                         2.00 Metallic   25490.00 25490.00 28495.00 33800.00  
                              Uni/basic  29200.00 29200.00 29200.00 29200.00  
                         3.00 Metallic   19900.00 21990.00 24912.50 27900.00  
                              Uni/basic  21447.50 21900.00 23445.00 24990.00  

[164 rows x 8 columns]

In [267]:
df["Paint_Type"].replace("-", np.nan, inplace = True)

In [268]:
fill(df, "make_model", "body_type", "Paint_Type", "ffill") #pearl effect fiyata etki edebilir,bu kategorical oldugu icin mode ve median ile dolduramam,
                                                            #

Number of NaN :  0
------------------
Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint_Type, dtype: int64


Type

In [269]:
first_looking("Type")            #2 tane nan var, 2 tanenin bir tanesi AudiA3  0 yasinda ve 3 yasinda

column name    :  Type
--------------------------------
per_of_nulls   :  % 0.01
num_of_nulls   :  2
num_of_uniques :  5
--------------------------------
Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64


In [270]:
df.Type.fillna("-", inplace=True)

In [271]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
-                     2
Name: Type, dtype: int64

In [272]:
df.groupby(["Type", "make_model", "age"]).km.describe()

count      mean      std       min  \
Type          make_model     age                                        
-             Audi A3        0.00   1.00   1647.36      NaN   1647.36   
                             3.00   1.00 115137.00      NaN 115137.00   
Demonstration Audi A1        0.00 110.00   3784.23  2498.41     10.00   
                             1.00  19.00   9931.16  5209.54   1050.00   
                             2.00   6.00  22963.50 17802.68  11940.00   
...                                  ...       ...      ...       ...   
Used          Renault Duster 0.00  16.00    101.00     0.00    101.00   
              Renault Espace 0.00  40.00   2101.03  3625.58     10.00   
                             1.00 160.00  16368.25 11001.74     10.00   
                             2.00 154.00  44076.11 27116.40   5600.00   
                             3.00 299.00  74481.99 34217.26   3532.00   

                                        25%       50%       75%       max  
Type          make_model     age                                           
-             Audi A3        0.00   1647.36   1647.36   1647.36   1647.36  
                             3.00 115137.00 115137.00 115137.00 115137.00  
Demonstration Audi A1        0.00   2541.25   3000.50   4999.75  12700.00  
                             1.00   5500.00   9522.00  14560.00  18900.00  
                             2.00  15327.25  16378.50  18425.00  59000.00  
...                                     ...       ...       ...       ...  
Used          Renault Duster 0.00    101.00    101.00    101.00    101.00  
              Renault Espace 0.00     10.00    100.00   2071.00  15400.00  
                             1.00   9345.50  15000.00  23002.50  62860.00  
                             2.00  25326.50  44677.50  49606.00 240000.00  
                             3.00  49950.00  68900.00  95000.00 197414.00  

[113 rows x 8 columns]

In [273]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [274]:
df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('-','New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('-','Used')

In [275]:
df['Type'].value_counts(dropna=False)

Used              11097
New                1651
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: Type, dtype: int64

Inspection_new

In [276]:
first_looking("Inspection_new")

column name    :  Inspection_new
--------------------------------
per_of_nulls   :  % 75.3
num_of_nulls   :  11987
num_of_uniques :  1
--------------------------------
NaN    11987
Yes     3932
Name: Inspection_new, dtype: int64


In [277]:
df["Inspection_new"].fillna("-", inplace=True)

In [278]:
df["Inspection_new"].value_counts(dropna=False)

-      11987
Yes     3932
Name: Inspection_new, dtype: int64

In [279]:
df.groupby(["make_model", "body_type", "age", "Inspection_new"]).price.describe()

count     mean     std      min  \
make_model     body_type age  Inspection_new                                    
Audi A1        Compact   0.00 -              243.00 24206.37 3530.09 16220.00   
                              Yes            133.00 22542.25 3076.12 14900.00   
                         1.00 -              150.00 19065.31 2836.07 14500.00   
                              Yes            118.00 17999.52 2293.78 13980.00   
                         2.00 -              115.00 16425.67 1985.79 10999.00   
...                                             ...      ...     ...      ...   
Renault Espace Van       1.00 Yes             33.00 33000.88 3414.04 29385.00   
                         2.00 -              137.00 26507.09 3422.05 14600.00   
                              Yes              2.00 27139.50  211.42 26990.00   
                         3.00 -              176.00 22182.27 3270.03 12990.00   
                              Yes             30.00 21790.10 3690.17 12614.00   

                                                  25%      50%      75%  \
make_model     body_type age  Inspection_new                              
Audi A1        Compact   0.00 -              21435.00 23480.00 27390.00   
                              Yes            20388.00 22400.00 22950.00   
                         1.00 -              16490.00 18800.00 21949.75   
                              Yes            16445.00 16910.00 19963.50   
                         2.00 -              15450.00 15850.00 16890.00   
...                                               ...      ...      ...   
Renault Espace Van       1.00 Yes            29990.00 31899.00 35300.00   
                         2.00 -              24980.00 25490.00 28500.00   
                              Yes            27064.75 27139.50 27214.25   
                         3.00 -              19900.00 22261.00 24550.00   
                              Yes            19085.00 21982.50 24825.00   

                                                  max  
make_model     body_type age  Inspection_new           
Audi A1        Compact   0.00 -              31990.00  
                              Yes            29197.00  
                         1.00 -              23777.00  
                              Yes            23829.00  
                         2.00 -              22150.00  
...                                               ...  
Renault Espace Van       1.00 Yes            38990.00  
                         2.00 -              33800.00  
                              Yes            27289.00  
                         3.00 -              32900.00  
                              Yes            26970.00  

[199 rows x 8 columns]

In [280]:
df["Inspection_new"].replace("-", "No", inplace=True)

In [281]:
df["Inspection_new"].value_counts(dropna=False)

No     11987
Yes     3932
Name: Inspection_new, dtype: int64

In [282]:
df["Inspection_new"].replace(["Yes", "No"], [1,0], inplace = True)

In [283]:
df["Inspection_new"].value_counts(dropna=False)

0    11987
1     3932
Name: Inspection_new, dtype: int64

Upholstery_type

In [284]:
first_looking("Upholstery_type")

column name    :  Upholstery_type
--------------------------------
per_of_nulls   :  % 30.6
num_of_nulls   :  4871
num_of_uniques :  5
--------------------------------
Cloth           8423
NaN             4871
Part leather    1499
Full leather    1009
Velour            60
alcantara         57
Name: Upholstery_type, dtype: int64


In [285]:
df.groupby(["make_model", "body_type", "age", "Upholstery_type"]).price.describe()

count     mean     std  \
make_model     body_type age  Upholstery_type                           
Audi A1        Compact   0.00 Cloth           310.00 23897.24 3306.37   
                              Full leather      1.00 20900.00     NaN   
                              Part leather      8.00 28760.00  275.53   
                              Velour           12.00 21617.50 3732.41   
                              alcantara         1.00 28960.00     NaN   
...                                              ...      ...     ...   
Renault Espace Van       2.00 Part leather     18.00 26697.28 3403.90   
                              alcantara         1.00 25499.00     NaN   
                         3.00 Cloth            20.00 20742.75 2889.78   
                              Full leather     65.00 23572.89 2874.26   
                              Part leather     45.00 21425.02 3477.98   

                                                   min      25%      50%  \
make_model     body_type age  Upholstery_type                              
Audi A1        Compact   0.00 Cloth           14900.00 21390.00 22865.00   
                              Full leather    20900.00 20900.00 20900.00   
                              Part leather    28280.00 28542.50 28910.00   
                              Velour          17330.00 19600.00 21135.00   
                              alcantara       28960.00 28960.00 28960.00   
...                                                ...      ...      ...   
Renault Espace Van       2.00 Part leather    21490.00 24217.50 27925.00   
                              alcantara       25499.00 25499.00 25499.00   
                         3.00 Cloth           16950.00 18305.00 20945.00   
                              Full leather    13500.00 21890.00 23900.00   
                              Part leather    12990.00 19500.00 21400.00   

                                                   75%      max  
make_model     body_type age  Upholstery_type                    
Audi A1        Compact   0.00 Cloth           27175.00 29197.00  
                              Full leather    20900.00 20900.00  
                              Part leather    28965.00 28990.00  
                              Velour          22490.00 31990.00  
                              alcantara       28960.00 28960.00  
...                                                ...      ...  
Renault Espace Van       2.00 Part leather    28147.50 33750.00  
                              alcantara       25499.00 25499.00  
                         3.00 Cloth           22472.50 27440.00  
                              Full leather    25900.00 27600.00  
                              Part leather    23990.00 27900.00  

[288 rows x 8 columns]

In [286]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], 
                              ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)
                                #bunlarin hepsi bir veri tipi diye düsündük

In [287]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth                8483
NaN                  4871
Part/Full Leather    2565
Name: Upholstery_type, dtype: int64

In [288]:
df["Upholstery_type"].fillna("-", inplace=True)

In [289]:
df.groupby(["make_model", "body_type", "age", "Upholstery_type"]).price.describe() #nanlar muhtemelen kumas diye düsündük

count     mean     std  \
make_model     body_type age  Upholstery_type                             
Audi A1        Compact   0.00 -                  44.00 21199.36 3026.85   
                              Cloth             322.00 23812.28 3344.83   
                              Part/Full Leather  10.00 27994.00 2505.18   
                         1.00 -                  52.00 18185.77 2696.11   
                              Cloth             194.00 18339.59 2548.07   
...                                                ...      ...     ...   
Renault Espace Van       2.00 Cloth              32.00 25667.41 1907.30   
                              Part/Full Leather  42.00 27886.17 3804.04   
                         3.00 -                  76.00 21665.30 3342.50   
                              Cloth              20.00 20742.75 2889.78   
                              Part/Full Leather 110.00 22694.22 3295.31   

                                                     min      25%      50%  \
make_model     body_type age  Upholstery_type                                
Audi A1        Compact   0.00 -                 16220.00 19104.25 20876.50   
                              Cloth             14900.00 21345.00 22757.50   
                              Part/Full Leather 20900.00 28507.50 28910.00   
                         1.00 -                 14500.00 16325.00 16820.00   
                              Cloth             13980.00 16445.00 16950.00   
...                                                  ...      ...      ...   
Renault Espace Van       2.00 Cloth             19450.00 25490.00 25490.00   
                              Part/Full Leather 20900.00 25283.00 28095.00   
                         3.00 -                 12614.00 18875.00 21985.00   
                              Cloth             16950.00 18305.00 20945.00   
                              Part/Full Leather 12990.00 20783.75 22990.00   

                                                     75%      max  
make_model     body_type age  Upholstery_type                      
Audi A1        Compact   0.00 -                 22993.00 29181.00  
                              Cloth             27027.50 31990.00  
                              Part/Full Leather 28960.00 28990.00  
                         1.00 -                 20541.00 23777.00  
                              Cloth             20487.50 23829.00  
...                                                  ...      ...  
Renault Espace Van       2.00 Cloth             25490.00 33800.00  
                              Part/Full Leather 30462.50 33750.00  
                         3.00 -                 23492.50 32900.00  
                              Cloth             22472.50 27440.00  
                              Part/Full Leather 24996.00 27900.00  

[291 rows x 8 columns]

In [290]:
df["Upholstery_type"].replace("-", np.nan, inplace=True)

In [291]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

Number of NaN :  0
------------------
Cloth                12238
Part/Full Leather     3681
Name: Upholstery_type, dtype: int64


Uphostery_color

In [292]:
df.drop("Upholstery_color", axis=1, inplace=True) #döseme rengi fiyati pek etkilemez diye düstük

Nr_of_Doors

In [293]:
first_looking("Nr_of_Doors")

column name    :  Nr_of_Doors
--------------------------------
per_of_nulls   :  % 1.33
num_of_nulls   :  212
num_of_uniques :  6
--------------------------------
5.00    11575
4.00     3079
3.00      832
2.00      219
NaN       212
1.00        1
7.00        1
Name: Nr_of_Doors, dtype: int64


In [294]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode") #bunlar sayisal ama mode kullandik,kapi sayisi ortalama alinacak bir deger degil 4,5 kapi yok
                                                      #bir sey bölünemiyorsa mean kullanilmaz o yüzden mode kullandik

Number of NaN :  0
------------------
5.00    11787
4.00     3079
3.00      832
2.00      219
1.00        1
7.00        1
Name: Nr_of_Doors, dtype: int64


Nr_of_Seats

In [295]:
first_looking("Nr_of_Seats")

column name    :  Nr_of_Seats
--------------------------------
per_of_nulls   :  % 6.14
num_of_nulls   :  977
num_of_uniques :  6
--------------------------------
5.00    13336
4.00     1125
NaN       977
7.00      362
2.00      116
6.00        2
3.00        1
Name: Nr_of_Seats, dtype: int64


In [296]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

Number of NaN :  0
------------------
5.00    14308
4.00     1127
7.00      362
2.00      119
6.00        2
3.00        1
Name: Nr_of_Seats, dtype: int64


Cylinders

In [297]:
first_looking("Cylinders")

column name    :  Cylinders
--------------------------------
per_of_nulls   :  % 35.68
num_of_nulls   :  5680
num_of_uniques :  7
--------------------------------
4.00    8105
NaN     5680
3.00    2104
5.00      22
6.00       3
8.00       2
2.00       2
1.00       1
Name: Cylinders, dtype: int64


In [298]:
df["Cylinders"].fillna("-", inplace=True)

In [299]:
df.groupby(["make_model", "body_type", "age", "Cylinders"]).price.describe()

count     mean     std      min  \
make_model     body_type age  Cylinders                                    
Audi A1        Compact   0.00 3.0       185.00 23789.51 3485.13 14900.00   
                              4.0        20.00 25508.05 3304.23 20582.00   
                              -         171.00 23210.80 3391.35 17780.00   
                         1.00 3.0       131.00 18609.00 2704.67 13980.00   
                              4.0        33.00 21175.70 2117.61 16100.00   
...                                        ...      ...     ...      ...   
Renault Espace Van       2.00 4.0        59.00 26812.37 4035.41 14600.00   
                              8.0         1.00 24990.00     NaN 24990.00   
                              -          79.00 26314.30 2860.47 17480.00   
                         3.00 4.0       134.00 22288.30 3188.10 13000.00   
                              -          72.00 21821.53 3576.38 12614.00   

                                             25%      50%      75%      max  
make_model     body_type age  Cylinders                                      
Audi A1        Compact   0.00 3.0       21435.00 22919.00 27380.00 28980.00  
                              4.0       22839.00 25578.00 28863.75 29181.00  
                              -         20388.00 22350.00 26492.00 31990.00  
                         1.00 3.0       16494.50 16980.00 21197.50 23700.00  
                              4.0       20500.00 21770.00 22750.00 23829.00  
...                                          ...      ...      ...      ...  
Renault Espace Van       2.00 4.0       24900.00 27400.00 28900.00 33800.00  
                              8.0       24990.00 24990.00 24990.00 24990.00  
                              -         24990.00 25490.00 27970.00 33285.00  
                         3.00 4.0       19900.00 21992.50 24900.00 27900.00  
                              -         19367.25 22540.00 23490.00 32900.00  

[277 rows x 8 columns]

In [300]:
df.drop("Cylinders", axis=1, inplace=True)  #silindir sayisi horse power ya da displacement ile ilgili,o yüzden bunu düstük
                                            #onlari kullanacagim zaten %36 missing value var

Drive_chain

In [301]:
first_looking("Drive_chain")

column name    :  Drive_chain
--------------------------------
per_of_nulls   :  % 43.08
num_of_nulls   :  6858
num_of_uniques :  3
--------------------------------
front    8886
NaN      6858
4WD       171
rear        4
Name: Drive_chain, dtype: int64


In [302]:
df["Drive_chain"].fillna("-", inplace=True)

In [303]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

count     mean      std      min  \
make_model     body_type   Drive_chain                                     
Audi A1        Compact     -           352.00 17620.87  4226.12 10490.00   
                           4WD           2.00 14790.00  1258.65 13900.00   
                           front       685.00 20008.22  4511.35  9950.00   
               Coupe       -             2.00 14925.00  1378.86 13950.00   
               Sedans      -           561.00 17830.44  4362.32  8999.00   
...                                       ...      ...      ...      ...   
Renault Espace Transporter 4WD           1.00 30000.00      NaN 30000.00   
                           front         6.00 23258.17  5973.32 17000.00   
               Van         -           394.00 29624.21  8543.18 12990.00   
                           4WD          38.00 38150.76 10013.78 24950.00   
                           front       379.00 31525.16  8343.19 12614.00   

                                            25%      50%      75%      max  
make_model     body_type   Drive_chain                                      
Audi A1        Compact     -           14990.00 15900.00 20885.75 29190.00  
                           4WD         14345.00 14790.00 15235.00 15680.00  
                           front       16430.00 19890.00 22690.00 31990.00  
               Coupe       -           14437.50 14925.00 15412.50 15900.00  
               Sedans      -           14900.00 16490.00 20700.00 37900.00  
...                                         ...      ...      ...      ...  
Renault Espace Transporter 4WD         30000.00 30000.00 30000.00 30000.00  
                           front       19262.50 21500.00 26624.25 32700.00  
               Van         -           23990.00 27545.00 34368.75 63477.00  
                           4WD         31900.00 34200.00 45747.00 57990.00  
                           front       25050.00 31360.00 36151.00 68320.00  

[93 rows x 8 columns]

In [304]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")
#renault duster 4 ceker o yüzden özel ilgilendik

In [305]:
df.loc[cond, 'Drive_chain'] = df.loc[cond, 'Drive_chain'].replace('-', '4WD')

In [306]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
-        6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [307]:
df["Drive_chain"] = df["Drive_chain"].replace('-', np.nan)

In [308]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [309]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: Drive_chain, dtype: int64


Emission_Class

In [310]:
first_looking("Emission_Class")

column name    :  Emission_Class
--------------------------------
per_of_nulls   :  % 22.79
num_of_nulls   :  3628
num_of_uniques :  3
--------------------------------
Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64


In [311]:
df["Emission_Class"].fillna("-", inplace=True)

In [312]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
-          3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [313]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

count     mean     std      min  \
make_model     age  Fuel    Emission_Class                                    
Audi A1        0.00 Benzine -              267.00 23770.32 3136.48 14900.00   
                            Euro 6         535.00 23948.53 3472.58 15550.00   
                    Diesel  Euro 6           4.00 19370.00 1181.89 17900.00   
               1.00 Benzine -               99.00 19317.06 2855.34 14490.00   
                            Euro 5           3.00 17862.67 1840.59 16800.00   
...                                           ...      ...     ...      ...   
Renault Espace 3.00 Benzine -                3.00 22790.00 3852.80 18480.00   
                            Euro 6          29.00 23773.41 2649.28 17900.00   
                    Diesel  -               65.00 22353.03 3158.17 14900.00   
                            Euro 5           1.00 24900.00     NaN 24900.00   
                            Euro 6         202.00 21425.39 3281.75 12614.00   

                                                25%      50%      75%      max  
make_model     age  Fuel    Emission_Class                                      
Audi A1        0.00 Benzine -              21875.00 22900.00 26330.00 31990.00  
                            Euro 6         21285.00 22900.00 27107.50 37900.00  
                    Diesel  Euro 6         18717.50 19495.00 20147.50 20590.00  
               1.00 Benzine -              16684.50 19887.00 21900.00 23880.00  
                            Euro 5         16800.00 16800.00 18394.00 19988.00  
...                                             ...      ...      ...      ...  
Renault Espace 3.00 Benzine -              21235.00 23990.00 24945.00 25900.00  
                            Euro 6         21950.00 23890.00 25990.00 27900.00  
                    Diesel  -              20745.00 22690.00 23990.00 32900.00  
                            Euro 5         24900.00 24900.00 24900.00 24900.00  
                            Euro 6         19034.75 20945.00 23950.00 30000.00  

[162 rows x 8 columns]

In [314]:
df["Emission_Class"].replace("-", np.nan, inplace=True)

In [315]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [316]:
fill(df, "age", "Fuel", "Emission_Class", "ffill")   #emission da sürekli farkli kurallar geliyor o yüzden age ve yakit i aldik
                                                     #2016 dan sonraki arclarda emission da pek sikinti yok, bu konuda bize yardimci olacak baska veri var ve düstük

Number of NaN :  0
------------------
Euro 6    15769
Euro 5      100
Euro 4       50
Name: Emission_Class, dtype: int64


In [317]:
df.drop("Emission_Class", axis=1, inplace=True)

GEARS

In [318]:
first_looking("Gears")

column name    :  Gears
--------------------------------
per_of_nulls   :  % 29.6
num_of_nulls   :  4712
num_of_uniques :  10
--------------------------------
6.00     5822
NaN      4712
5.00     3239
7.00     1908
8.00      224
9.00        6
1.00        2
3.00        2
4.00        2
2.00        1
50.00       1
Name: Gears, dtype: int64


In [319]:
df["Gears"].fillna("-", inplace=True)

In [320]:
df.groupby(["make_model", "body_type", "Gearing_Type", "Gears"]).price.describe()

count     mean      std  \
make_model     body_type Gearing_Type   Gears                            
Audi A1        Compact   Automatic      5.0     3.00 22184.33  3421.30   
                                        6.0     6.00 21038.33  4282.04   
                                        7.0   199.00 22059.25  3918.12   
                                        8.0     1.00 16880.00      NaN   
                                        -     253.00 21640.43  4965.50   
...                                              ...      ...      ...   
Renault Espace Van       Manual         7.0     1.00 36590.00      NaN   
                                        -      14.00 27123.21  8657.07   
                         Semi-automatic 6.0    11.00 28885.45 11439.11   
                                        7.0     1.00 19900.00      NaN   
                                        -       4.00 26445.00  4549.29   

                                                   min      25%      50%  \
make_model     body_type Gearing_Type   Gears                              
Audi A1        Compact   Automatic      5.0   18497.00 20648.50 22800.00   
                                        6.0   16430.00 18725.00 20920.00   
                                        7.0   13990.00 18970.00 21790.00   
                                        8.0   16880.00 16880.00 16880.00   
                                        -     13880.00 16975.00 20950.00   
...                                                ...      ...      ...   
Renault Espace Van       Manual         7.0   36590.00 36590.00 36590.00   
                                        -     16480.00 19623.00 25740.00   
                         Semi-automatic 6.0   16900.00 21450.00 24900.00   
                                        7.0   19900.00 19900.00 19900.00   
                                        -     22900.00 23650.00 24945.00   

                                                   75%      max  
make_model     body_type Gearing_Type   Gears                    
Audi A1        Compact   Automatic      5.0   24028.00 25256.00  
                                        6.0   21060.00 28860.00  
                                        7.0   24365.00 29181.00  
                                        8.0   16880.00 16880.00  
                                        -     26980.00 29197.00  
...                                                ...      ...  
Renault Espace Van       Manual         7.0   36590.00 36590.00  
                                        -     32625.00 43080.00  
                         Semi-automatic 6.0   32750.00 56950.00  
                                        7.0   19900.00 19900.00  
                                        -     27740.00 32990.00  

[215 rows x 8 columns]

In [321]:
df["Gears"].replace("-", np.nan, inplace=True)

In [322]:
for group1 in df["make_model"].unique():
    for group2 in df["body_type"].unique():
        for group3 in df["Gearing_Type"].unique():
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing_Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

In [323]:
df["Gears"].value_counts(dropna=False)

6.00     8615
5.00     4255
7.00     2810
8.00      225
9.00        6
1.00        2
3.00        2
4.00        2
2.00        1
50.00       1
Name: Gears, dtype: int64

hp_kW

In [324]:
first_looking("hp_kW")

column name    :  hp_kW
--------------------------------
per_of_nulls   :  % 0.55
num_of_nulls   :  88
num_of_uniques :  80
--------------------------------
85.00     2542
66.00     2122
81.00     1402
100.00    1308
110.00    1112
          ... 
84.00        1
195.00       1
44.00        1
239.00       1
9.00         1
Name: hp_kW, Length: 81, dtype: int64


In [325]:
df["hp_kW"].fillna("-", inplace=True)

In [326]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

count     mean     std      min      25%  \
make_model     body_type hp_kW                                             
Audi A1        Compact   60.0   58.00 15189.83 1663.61 10900.00 14390.00   
                         66.0  162.00 15398.21 1934.30 10490.00 14042.50   
                         70.0  332.00 17983.26 3315.81  9950.00 15480.00   
                         71.0   32.00 20831.59 2410.84 15890.00 18937.50   
                         85.0  330.00 22604.10 4572.57 11100.00 19700.00   
...                               ...      ...     ...      ...      ...   
Renault Espace Van       150.0   1.00 43545.00     NaN 43545.00 43545.00   
                         165.0 175.00 38205.96 7911.48 26800.00 31990.00   
                         167.0   2.00 37990.00 2828.43 35990.00 36990.00   
                         168.0   3.00 38410.00 1712.66 36590.00 37620.00   
                         -      34.00 28466.03 4898.73 17990.00 24515.00   

                                    50%      75%      max  
make_model     body_type hp_kW                             
Audi A1        Compact   60.0  15774.50 16345.00 16978.00  
                         66.0  15465.00 16445.00 23700.00  
                         70.0  17447.00 21190.00 28990.00  
                         71.0  21425.00 22462.25 25256.00  
                         85.0  22497.00 26980.00 31990.00  
...                                 ...      ...      ...  
Renault Espace Van       150.0 43545.00 43545.00 43545.00  
                         165.0 35989.00 40999.00 64332.00  
                         167.0 37990.00 38990.00 39990.00  
                         168.0 38650.00 39320.00 39990.00  
                         -     28195.00 32900.00 39990.00  

[311 rows x 8 columns]

In [327]:
df["hp_kW"].replace(["-"], np.nan, inplace=True)

In [328]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

Number of NaN :  0
------------------
85.00     2543
66.00     2124
81.00     1403
100.00    1314
110.00    1113
          ... 
123.00       1
44.00        1
195.00       1
75.00        1
9.00         1
Name: hp_kW, Length: 80, dtype: int64


Displacement_cc

In [329]:
first_looking("Displacement_cc")

column name    :  Displacement_cc
--------------------------------
per_of_nulls   :  % 3.12
num_of_nulls   :  496
num_of_uniques :  77
--------------------------------
1598.00    4761
999.00     2438
1398.00    1314
1399.00     749
1229.00     677
           ... 
122.00        1
1198.00       1
1195.00       1
2967.00       1
1800.00       1
Name: Displacement_cc, Length: 78, dtype: int64


In [330]:
df["Displacement_cc"].fillna("-", inplace=True)

In [331]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

count     mean     std      min  \
make_model     body_type Displacement_cc                                    
Audi A1        Compact   929.0             5.00 16364.00 1504.82 14980.00   
                         995.0             2.00 21990.00 1414.21 20990.00   
                         999.0           663.00 20294.10 4570.80  9950.00   
                         1000.0            7.00 19498.43 5396.20 13500.00   
                         1395.0           98.00 18246.62 3084.08 12550.00   
...                                         ...      ...     ...      ...   
Renault Espace Van       1798.0          174.00 38173.30 7873.21 26800.00   
                         1800.0            1.00 50750.00     NaN 50750.00   
                         1997.0           92.00 39621.42 7514.27 12990.00   
                         2000.0            1.00 21900.00     NaN 21900.00   
                         -                77.00 28619.61 5732.77 17500.00   

                                              25%      50%      75%      max  
make_model     body_type Displacement_cc                                      
Audi A1        Compact   929.0           15480.00 16240.00 16240.00 18880.00  
                         995.0           21490.00 21990.00 22490.00 22990.00  
                         999.0           16480.00 20380.00 22820.00 31990.00  
                         1000.0          16024.50 16800.00 22695.00 28750.00  
                         1395.0          15850.00 17835.00 20900.00 28880.00  
...                                           ...      ...      ...      ...  
Renault Espace Van       1798.0          31990.00 35990.00 40991.75 64332.00  
                         1800.0          50750.00 50750.00 50750.00 50750.00  
                         1997.0          32975.00 39625.00 46262.50 56950.00  
                         2000.0          21900.00 21900.00 21900.00 21900.00  
                         -               24490.00 28465.00 32935.00 44500.00  

[289 rows x 8 columns]

In [332]:
df["Displacement_cc"].replace("-", np.nan, inplace=True)

In [333]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

Number of NaN :  0
------------------
1598.00    5044
999.00     2467
1398.00    1387
1399.00     768
1229.00     678
           ... 
122.00        1
1198.00       1
1195.00       1
2967.00       1
1800.00       1
Name: Displacement_cc, Length: 77, dtype: int64


Weight_kg

In [334]:
first_looking("Weight_kg")

column name    :  Weight_kg
--------------------------------
per_of_nulls   :  % 43.81
num_of_nulls   :  6974
num_of_uniques :  434
--------------------------------
NaN        6974
1163.00     574
1360.00     356
1165.00     301
1335.00     242
           ... 
1960.00       1
1258.00       1
1167.00       1
1331.00       1
2037.00       1
Name: Weight_kg, Length: 435, dtype: int64


In [335]:
df["Weight_kg"].fillna("-", inplace=True)

In [336]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

count     mean     std      min      25%  \
make_model     body_type Weight_kg                                             
Audi A1        Compact   102.0       1.00 19229.00     NaN 19229.00 19229.00   
                         1010.0      2.00 15450.00  707.11 14950.00 15200.00   
                         1035.0      6.00 16796.67 2617.87 14390.00 15892.50   
                         1040.0      2.00 20424.50 2933.79 18350.00 19387.25   
                         1065.0     36.00 20971.78 1982.55 15500.00 18987.50   
...                                   ...      ...     ...      ...      ...   
Renault Espace Van       2037.0      1.00 47950.00     NaN 47950.00 47950.00   
                         2353.0      1.00 22990.00     NaN 22990.00 22990.00   
                         2410.0      1.00 23990.00     NaN 23990.00 23990.00   
                         2471.0      5.00 24738.00 8470.64 17400.00 20900.00   
                         -         377.00 30126.30 7273.48 12614.00 24990.00   

                                        50%      75%      max  
make_model     body_type Weight_kg                             
Audi A1        Compact   102.0     19229.00 19229.00 19229.00  
                         1010.0    15450.00 15700.00 15950.00  
                         1035.0    15900.00 16575.00 21900.00  
                         1040.0    20424.50 21461.75 22499.00  
                         1065.0    21690.00 22400.00 23550.00  
...                                     ...      ...      ...  
Renault Espace Van       2037.0    47950.00 47950.00 47950.00  
                         2353.0    22990.00 22990.00 22990.00  
                         2410.0    23990.00 23990.00 23990.00  
                         2471.0    20900.00 25500.00 38990.00  
                         -         29990.00 34950.00 68320.00  

[894 rows x 8 columns]

In [337]:
df["Weight_kg"].replace("-", np.nan, inplace=True)

In [338]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

Number of NaN :  0
------------------
1163.00    1582
1360.00    1419
1487.00     966
1135.00     837
1425.00     744
           ... 
1331.00       1
1132.00       1
1252.00       1
1792.00       1
2037.00       1
Name: Weight_kg, Length: 434, dtype: int64


CO2 Emission

In [339]:
first_looking("CO2_Emission")

column name    :  CO2_Emission
--------------------------------
per_of_nulls   :  % 15.3
num_of_nulls   :  2436
num_of_uniques :  122
--------------------------------
NaN        2436
120.00      740
99.00       545
97.00       537
104.00      501
           ... 
14.00         1
1060.00       1
51.00         1
165.00        1
193.00        1
Name: CO2_Emission, Length: 123, dtype: int64


In [340]:
df["CO2_Emission"].fillna("-", inplace=True)

In [341]:
df.groupby(["make_model", "body_type", "Fuel", "CO2_Emission"]).price.describe()

count     mean     std  \
make_model     body_type Fuel    CO2_Emission                           
Audi A1        Compact   Benzine 97.0          77.00 15115.64 1746.38   
                                 98.0          23.00 17077.57 3010.23   
                                 100.0          3.00 15903.33 1859.90   
                                 102.0        106.00 17401.34 3100.21   
                                 103.0         30.00 21233.03 1794.55   
...                                              ...      ...     ...   
Renault Espace Van       Diesel  168.0          3.00 46723.00 1745.90   
                                 172.0          1.00 48800.00     NaN   
                                 174.0          1.00 43490.00     NaN   
                                 191.0          2.00 28465.00    0.00   
                                 -            176.00 29169.26 6754.36   

                                                   min      25%      50%  \
make_model     body_type Fuel    CO2_Emission                              
Audi A1        Compact   Benzine 97.0         10900.00 13820.00 15444.00   
                                 98.0         13999.00 15290.00 16100.00   
                                 100.0        14220.00 14905.00 15590.00   
                                 102.0         9950.00 15772.50 16950.00   
                                 103.0        18350.00 19225.00 22189.50   
...                                                ...      ...      ...   
Renault Espace Van       Diesel  168.0        44999.00 45839.50 46680.00   
                                 172.0        48800.00 48800.00 48800.00   
                                 174.0        43490.00 43490.00 43490.00   
                                 191.0        28465.00 28465.00 28465.00   
                                 -            12990.00 24372.50 28675.00   

                                                   75%      max  
make_model     body_type Fuel    CO2_Emission                    
Audi A1        Compact   Benzine 97.0         16669.00 18775.00  
                                 98.0         16900.00 25256.00  
                                 100.0        16745.00 17900.00  
                                 102.0        19889.25 29150.00  
                                 103.0        22448.50 23550.00  
...                                                ...      ...  
Renault Espace Van       Diesel  168.0        47585.00 48490.00  
                                 172.0        48800.00 48800.00  
                                 174.0        43490.00 43490.00  
                                 191.0        28465.00 28465.00  
                                 -            32900.00 56950.00  

[946 rows x 8 columns]

In [342]:
df.groupby(["Fuel", "make_model", "body_type", "CO2_Emission"]).price.describe().loc["Electric"]

count     mean     std      min      25%  \
make_model   body_type CO2_Emission                                             
Audi A3      Compact   -              1.00 24950.00     NaN 24950.00 24950.00   
             Sedans    36.0           3.00 37449.67 1361.64 35899.00 36949.50   
Renault Clio Sedans    -              1.00  8490.00     NaN  8490.00  8490.00   

                                         50%      75%      max  
make_model   body_type CO2_Emission                             
Audi A3      Compact   -            24950.00 24950.00 24950.00  
             Sedans    36.0         38000.00 38225.00 38450.00  
Renault Clio Sedans    -             8490.00  8490.00  8490.00

In [343]:
df["CO2_Emission"].replace("-", np.nan, inplace=True)

In [344]:
df.loc[df.Fuel == "Electric", "CO2_Emission"]

3356      NaN
3612    36.00
3615    36.00
3617    36.00
13397     NaN
Name: CO2_Emission, dtype: float64

In [345]:
df.loc[df.Fuel == "Electric", "CO2_Emission"] = df.loc[df.Fuel == "Electric", "CO2_Emission"].fillna(df.loc[df.Fuel == "Electric", "CO2_Emission"].mode()[0])
df.loc[df.Fuel == "Electric", "CO2_Emission"]

3356    36.00
3612    36.00
3615    36.00
3617    36.00
13397   36.00
Name: CO2_Emission, dtype: float64

In [346]:
df['CO2_Emission'].fillna(df.groupby(["make_model", "body_type", "Fuel"])["CO2_Emission"].transform("median"), inplace = True)
df['CO2_Emission'].fillna(df.groupby(["make_model", "body_type"])["CO2_Emission"].transform("median"), inplace = True)
df['CO2_Emission'].fillna(df.groupby("make_model")["CO2_Emission"].transform("median"), inplace = True)
df['CO2_Emission'].fillna(df["CO2_Emission"].median(), inplace = True)

In [347]:
df["CO2_Emission"].value_counts(dropna=False)

120.00    1000
104.00     782
97.00      631
99.00      593
124.00     574
          ... 
80.00        1
14.00        1
51.00        1
165.00       1
193.00       1
Name: CO2_Emission, Length: 124, dtype: int64

In [348]:
def fill_median(df, group_col1, group_col2, group_col3, col_name):
    
    df[col_name].fillna(df.groupby([group_col1, group_col2, group_col3])[col_name].transform("median"), inplace = True)
    df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
    df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
    df[col_name].fillna(df[col_name].median(), inplace = True)

    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))  

In [349]:
fill_median(df, "make_model", "body_type", "Fuel", "CO2_Emission")

Number of NaN :  0
------------------
120.00    1000
104.00     782
97.00      631
99.00      593
124.00     574
          ... 
80.00        1
14.00        1
51.00        1
165.00       1
193.00       1
Name: CO2_Emission, Length: 124, dtype: int64


Comfort_Convenience

In [350]:
first_looking("Comfort_Convenience")

column name    :  Comfort_Convenience
--------------------------------
per_of_nulls   :  % 5.78
num_of_nulls   :  920
num_of_uniques :  6198
--------------------------------
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                               

In [351]:
fill(df, "make_model", "body_type", "Comfort_Convenience", "mode")

Number of NaN :  0
------------------
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                                   388
Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system                                                                                                                                       

Entertainment_Media

In [352]:
first_looking("Entertainment_Media")

column name    :  Entertainment_Media
--------------------------------
per_of_nulls   :  % 8.63
num_of_nulls   :  1374
num_of_uniques :  346
--------------------------------
NaN                                                                                                 1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                          1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                       982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                             783
On-board computer,Radio                                                                              487
                                                                                                    ... 
Bluetooth,CD player,Digital radio,Hands-free equipment,Radio                                           1
CD player,Hands-free equipment,MP3,Radio,USB                                               

In [353]:
fill(df, "make_model", "body_type", "Entertainment_Media", "mode")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                      1738
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                  1134
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB        1010
On-board computer                                                                615
Radio                                                                            558
                                                                                ... 
Bluetooth,CD player,MP3                                                            1
CD player,USB                                                                      1
Bluetooth,CD player,Digital radio,Radio,USB                                        1
Bluetooth,CD player,Digital radio,MP3,On-board computer,Radio,Television,USB       1
Hands-free equipment,On-board computer,Radio,Sound system                          1
Name: Entertainment_Media, 

Extras

In [354]:
first_looking("Extras")

column name    :  Extras
--------------------------------
per_of_nulls   :  % 18.61
num_of_nulls   :  2962
num_of_uniques :  659
--------------------------------
Alloy wheels                                                                                               3245
NaN                                                                                                        2962
Alloy wheels,Touch screen                                                                                   697
Alloy wheels,Voice Control                                                                                  577
Alloy wheels,Touch screen,Voice Control                                                                     541
                                                                                                           ... 
Alloy wheels,Catalytic Converter,Shift paddles,Sport package,Sport seats,Sport suspension,Voice Control       1
Alloy wheels,Catalytic Converter,Roof rack,Sport packa

In [355]:
fill(df, "make_model", "body_type", "Extras", "mode")

Number of NaN :  0
------------------
Alloy wheels                                                                                               5786
Alloy wheels,Touch screen                                                                                   697
Roof rack                                                                                                   596
Alloy wheels,Voice Control                                                                                  582
Alloy wheels,Touch screen,Voice Control                                                                     544
                                                                                                           ... 
Alloy wheels,Catalytic Converter,Shift paddles,Sport package,Sport seats,Sport suspension,Voice Control       1
Alloy wheels,Catalytic Converter,Roof rack,Sport package,Sport seats,Trailer hitch                            1
Alloy wheels,Catalytic Converter,Ski bag,Sport package,Voice Contr

Safety_Security

In [356]:
first_looking("Safety_Security")

column name    :  Safety_Security
--------------------------------
per_of_nulls   :  % 6.17
num_of_nulls   :  982
num_of_uniques :  4443
--------------------------------
NaN                                                                                                                                                                                                                                                                                                                                                               982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                      538
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Pa

In [357]:
fill(df, "make_model", "body_type", "Safety_Security", "mode")

Number of NaN :  0
------------------
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                      729
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                                 480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction contro

cons_comb

In [358]:
first_looking("cons_comb")

column name    :  cons_comb
--------------------------------
per_of_nulls   :  % 12.77
num_of_nulls   :  2033
num_of_uniques :  72
--------------------------------
NaN      2033
5.40      770
3.90      733
4.00      713
5.10      657
         ... 
33.00       1
50.00       1
1.00        1
46.00       1
11.00       1
Name: cons_comb, Length: 73, dtype: int64


In [359]:
cons_comb_2 = (df["cons_country"] + df["cons_city"])/2
cons_comb_2.value_counts(dropna=False)

NaN      2447
3.90      610
5.65      562
4.75      501
4.10      440
         ... 
7.45        1
1.50        1
12.05       1
9.55        1
7.25        1
Length: 160, dtype: int64

In [360]:
df["cons_comb"] = df["cons_comb"].fillna(cons_comb_2)

In [361]:
df["cons_comb"].value_counts(dropna=False)

NaN      1925
5.40      770
3.90      733
4.00      713
5.10      657
         ... 
33.00       1
50.00       1
1.00        1
46.00       1
13.80       1
Name: cons_comb, Length: 79, dtype: int64

In [362]:
df["cons_comb"].fillna("-", inplace=True)

In [363]:
df.groupby(["make_model", "body_type", "Fuel", "cons_comb"]).price.describe()

count     mean     std      min  \
make_model     body_type Fuel    cons_comb                                    
Audi A1        Compact   Benzine 4.0        25.00 18427.20 2538.25 12200.00   
                                 4.2        87.00 15631.13 2420.35 10900.00   
                                 4.3        17.00 16628.71 2467.67 13999.00   
                                 4.4       193.00 17618.92 3250.85  9950.00   
                                 4.5        11.00 21482.91 1918.66 18400.00   
...                                           ...      ...     ...      ...   
Renault Espace Van       Diesel  6.0         1.00 35100.00     NaN 35100.00   
                                 6.2         1.00 20745.00     NaN 20745.00   
                                 6.5         1.00 68320.00     NaN 68320.00   
                                 46.0        1.00 25490.00     NaN 25490.00   
                                 -         126.00 29010.91 6022.65 12990.00   

                                                25%      50%      75%      max  
make_model     body_type Fuel    cons_comb                                      
Audi A1        Compact   Benzine 4.0       16500.00 18670.00 19900.00 22919.00  
                                 4.2       13970.00 15599.00 16822.50 25256.00  
                                 4.3       15290.00 15900.00 16900.00 22919.00  
                                 4.4       15689.00 16950.00 19990.00 28750.00  
                                 4.5       19744.00 22444.00 23075.00 23250.00  
...                                             ...      ...      ...      ...  
Renault Espace Van       Diesel  6.0       35100.00 35100.00 35100.00 35100.00  
                                 6.2       20745.00 20745.00 20745.00 20745.00  
                                 6.5       68320.00 68320.00 68320.00 68320.00  
                                 46.0      25490.00 25490.00 25490.00 25490.00  
                                 -         24257.50 29244.50 32900.00 47950.00  

[711 rows x 8 columns]

In [364]:
df.groupby(["Fuel", "make_model", "body_type", "cons_comb"]).price.describe().loc["Electric"]

count     mean     std      min      25%  \
make_model   body_type cons_comb                                             
Audi A3      Compact   -           1.00 24950.00     NaN 24950.00 24950.00   
             Sedans    1.6         3.00 37449.67 1361.64 35899.00 36949.50   
Renault Clio Sedans    -           1.00  8490.00     NaN  8490.00  8490.00   

                                      50%      75%      max  
make_model   body_type cons_comb                             
Audi A3      Compact   -         24950.00 24950.00 24950.00  
             Sedans    1.6       38000.00 38225.00 38450.00  
Renault Clio Sedans    -          8490.00  8490.00  8490.00

In [365]:
df["cons_comb"].replace([0.0, 1.0, 1.2, 1.6, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [366]:
df.loc[df.Fuel == "Electric", "cons_comb"]

3356    NaN
3612    NaN
3615    NaN
3617    NaN
13397   NaN
Name: cons_comb, dtype: float64

In [367]:
df["cons_comb"].value_counts(dropna=False)

NaN     1983
5.40     770
3.90     733
4.00     713
5.10     657
        ... 
5.45       2
8.10       2
7.50       1
9.10       1
4.55       1
Name: cons_comb, Length: 62, dtype: int64

In [368]:
fill(df, "make_model", "body_type", "cons_comb", "median")

Number of NaN :  0
------------------
4.00    1020
5.60     965
5.40     900
5.00     810
4.70     766
        ... 
8.10       2
4.55       2
7.50       1
9.10       1
4.30       1
Name: cons_comb, Length: 63, dtype: int64


In [369]:
df.drop("cons_country", axis = 1, inplace = True)

In [370]:
df.drop("cons_city", axis = 1, inplace = True)

In [371]:
df.shape

(15919, 25)

In [372]:
df.isnull().sum()*100 / df.shape[0]

make_model            0.00
body_type             0.00
price                 0.00
km                    0.00
Type                  0.00
Fuel                  0.00
Comfort_Convenience   0.00
Entertainment_Media   0.00
Extras                0.00
Safety_Security       0.00
Gears                 0.00
age                   0.00
Previous_Owners       0.00
hp_kW                 0.00
Inspection_new        0.00
Paint_Type            0.00
Upholstery_type       0.00
Nr_of_Doors           0.00
Nr_of_Seats           0.00
Gearing_Type          0.00
Displacement_cc       0.00
Weight_kg             0.00
Drive_chain           0.00
cons_comb             0.00
CO2_Emission          0.00
dtype: float64

In [373]:
df.to_csv("filled_scout.csv", index=False)